# SNS 서비스 Shortletters

"Shortletters"는 게시글 위주의 SNS로서 게시글에 자유롭게 해시태그를 달 수 있고 해시태그를 기준으로 게시글을 찾아볼 수 있는 서비스

## MVP?

Minimum Viable Product 의 줄임말로 자유롭게 해시트개를 달 수 있는 게시글과 해시태그를 기준으로 데이터를 모아서 확인할 수 있는 것이다.
<br>
전체 SNS는 *MSA*로 이루어져 있으므로 유저 정보 등은 이번 프로젝트에서 따로 고민하지 않아도 된다.<br>
* MSA(Micro Service Architecture)

## 기술 선택

각 해시태그를 기준으로 게시글들을 모아볼 수 있어야 함으로 해시태그를 기준으로도 각 문서가 인덱싱되어야 한다.

# 개념적 설계 단계

NoSQL은 RDB에 비해 상대적으로 자유로운 스키마를 추구하고 논리적/물리적 설계 단계에서의 성능향상에 관심을 가진다.
<br>
<br>
## 개념적 설계 순서


엔터티 식별 -> 관계 식별 -> 속성 식별 <br>
<br>
### 엔터티 식별

게시글 MSA의 MVP에서는 게시글 collection만 다룬다.<br>
### 관계 식별

MongoDB에서도 collection간에 참조 가능<br>
### 속성 식별

자료의 속성이 계층형이다.<br>
RDB에서 사용하던 ERD로는 표현이 어렵다.

# 논리적 설계 단계
### NoSQL의 논리적 설계는 필요 없는가?
자료가 아닌 정보로 사용하기 위해서 스키마 설계는 필요하다.<br>
schemaless하다는 것은 자료로써 사용 가능성을 열어두는 것<br>

## 논리적 설계의 꽃, 인덱스 설계

NoSQL은 태생부터 빅데이터를 염두에 두고 탄생했기에 **Volume, Velocity, Variety**에 대응하기 위해 인덱싱은 필수적이다.<br>

#### 단일키 인덱스 설계 예시
```python
letters.create_index([("hashTags", pymongo.ASCENDING)]
```

#### 복합키 인덱스 설계 예시
```python
letters.create_index([("createAt", pymongo.DESCENDING),("_id", pymongo.DESCENDING)]
```

# 물리적 설계 단계

## 물리적 설계 단계에서는 무엇을 해야하는가?

파티셔닝, 클러스터 컴퓨팅, 샤딩, 레플리카

# 실습 - 인덱스 생성하기

지시사항

1. shortletters 데이터베이스를 생성하세요.

2. letters collection을 만들고 insert_many method를 활용하여 데이터를 삽입하세요.

3. 아래 내용을 참고하여 인덱스를 생성하세요.
- 단일 키 인덱스를 생성하세요. hashTags 필드에 대해 오름차순 정렬을 해보세요.
- 복합 키 인덱스를 생성하세요. createAt, _id 필드에 대해 내림차순 정렬을 해보세요

```python
# 문제풀이를 위해 기본적으로 제공되는 코드입니다. 수정하지 마세요!
import json
import sys
import pymongo
# 미리 준비된 샘플 데이터를 읽어오는 코드입니다.
with open("sample_data.json") as f:
    sample_data = json.load(f)
# 데이터베이스를 연결하는 코드입니다.
conn = pymongo.MongoClient("mongodb://localhost:27017/")
# 지시사항을 참고하여 shortletters 데이터베이스를 생성하세요.
db_shortletters = conn.get_database("shortletters")
# 지시사항을 참고하여 `letters` collection을 만들고 데이터를 삽입하세요.
col_letters = db_shortletters.get_collection("letters")
col_letters.insert_many(sample_data)
# 지시사항을 참고하여 인덱스를 생성하세요.
col_letters.create_index([("hashTags", pymongo.ASCENDING)])
col_letters.create_index([
    ("createAt", pymongo.DESCENDING),
    ("_id", pymongo.DESCENDING)
])
# 아래의 코드는 인덱스가 잘 생성되었는지 확인하기 위한 코드입니다. 수정하지 마세요!
result = col_letters.index_information()
print(result)
```

# 실습 - shortletters 뉴스피드와 해시태그모아보기

지시사항
1. MongoDB: shortletters 데이터베이스를 생성하세요.

2. MongoDB: letters collection을 만들고 데이터를 삽입하세요. insert_many method를 사용해서 샘플 데이터를 삽입합니다.

3. 장식자를 이용해 / URL에서 index.html을 출력하는 index() 메소드를 만드세요. 이때, MongoDB로 letters collection의 전체 데이터를 읽어와서 파라미터로 넘겨줍니다. 해당 페이지는 shortletters news feeds를 출력합니다.

4. 장식자를 이용해 hashtag/<hashtag> URL에서 hashtag.html을 출력하는 get_hash_tag() 메소드를 만드세요. 이때, MongoDB에서 <hashtag>로 탐색한 결과를 파라미터로 넘겨줍니다. 해당 페이지는 해시태그 모아보기 정보를 출력합니다.

```python
# 아래 코드는 Flask, PyMongo 환경을 구성하는 코드입니다. 수정하지 마세요!
import json
from flask import Flask, render_template
import pymongo

app = Flask(__name__)

# 샘플 데이터를 읽어오는 코드입니다.
with open("sample_data.json") as f:
    sample_data = json.load(f)

# 데이터베이스를 연결하는 코드입니다.
conn = pymongo.MongoClient("mongodb://localhost:27017/")


# 지시사항을 참고하여 shortletters 데이터베이스를 만드세요.
db_shortletters = conn.get_database("shortletters")


# 지시사항을 참고하여 letters collection을 만들고 데이터를 삽입하세요.
col_letters = db_shortletters.get_collection("letters")
col_letters.insert_many(sample_data)


# 지시사항을 참고하여 shortletters news feeds를 출력합니다.
@app.route("/", methods=["GET"])
def index():
    letters = col_letters.find({})
    return render_template("index.html", input_data=letters)


# 지시사항을 참고하여 해시태그 모아보기 정보를 출력합니다.
@app.route("/hashtag/<hashtag>", methods=["GET"])
def get_hash_tag(hashtag):
    letters=col_letters.find({"hashTags": hashtag})
    return render_template("hashtag.html", hash_tag=hashtag, input_data=letters)




```